In [24]:
from pyspark.sql import SparkSession

#Starting the sparks session

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .appName("read_weather_data")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [25]:
#B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.87:9000/parking-citations.csv')\
    .cache()
data_frame.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

In [26]:
#B.2 Count the number of partitions in the underlying RDD. 
data_frame.rdd.getNumPartitions()

11

In [27]:
#B.3 Print the schema for the DataFrame. 
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [28]:
#B.4 Count the number of rows in the CSV file
data_frame.count()

9881842

In [29]:
#B.5 Drop the columns Agency Description, Agency, and Route. 
data_frame = data_frame.drop("Agency Description", "Agency","Route")

In [30]:
#B.6 Find the mean fine amount (you need to convert the column to a float). 
from pyspark.sql.functions import col, avg
float_dataframe=data_frame.withColumn("Fine amount", data_frame["Fine amount"].cast("double"))
float_dataframe.select(avg(float_dataframe['Fine amount'])).show()


+----------------+
|avg(Fine amount)|
+----------------+
|70.1855354220642|
+----------------+



In [31]:
#B.7 Show the top 10 most frequent vehicle makes, and their frequencies
data_frame.groupBy("Make").count().sort(col("count").desc()).show(10)

+----+-------+
|Make|  count|
+----+-------+
|TOYT|1633266|
|HOND|1113834|
|FORD| 860828|
|NISS| 709250|
|CHEV| 674422|
| BMW| 450909|
|MERZ| 402126|
|VOLK| 335618|
|HYUN| 304934|
|DODG| 290979|
+----+-------+
only showing top 10 rows



In [32]:


COLORS = { 'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black', 'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze', 'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold', 'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory', 'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon', 'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver', 'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown' }
from pyspark.sql.functions import col, create_map, lit
from itertools import chain
def colourCount(df):
    mapping_expr = create_map([lit(x) for x in chain(*COLORS.items())])
    df.withColumn("ColorLong", mapping_expr[col("Color")]).where(df.Make== "HOND").groupBy("ColorLong").count().sort(col("count").desc()).show(10)

In [33]:
colourCount(data_frame)

+---------+------+
|ColorLong| count|
+---------+------+
|     Gray|266135|
|    Black|234627|
|   Silver|180219|
|    White|160165|
|     Blue|100208|
|      Red| 46802|
|    Green| 45293|
|     Gold| 17679|
|     null| 17675|
|   Maroon| 17079|
+---------+------+
only showing top 10 rows



In [34]:
# release the cores for another application!
spark_context.stop()